In [13]:
import os
import torch
import transformers
import pandas as pd
from datasets import load_from_disk
from datasets import load_dataset, Dataset, concatenate_datasets

In [4]:
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer

In [5]:
BASE_MODEL = "yanolja/EEVE-Korean-10.8B-v1.0"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [16]:
sample = pd.read_csv("sample_finetuning_data.csv", index_col =0)

In [18]:
sample.drop(columns = ['Tokenized_sentence', 'Tokenized_sentence_list'], inplace = True)

In [21]:
sample.reset_index(drop = True)

,add_rule_based_split_review,Tokenized_morphs
0,베트남이라 일반 콘센트도 가능했지만 이참에 하나 장만했어요,베트남 일반 콘센트
1,디자인도 깔끔해서 마음에 들고 성능도 확실해요,디자인 성능
2,일행이 다이소 제품 가져와서 비교해 봤는데 제거 80 충전될 때 동행은 40 충전...,비교 제거 충전 동행 충전
3,디자인도 예쁜데 케이스도 있어서 너무 좋습니다,디자인 케이스
4,국내에서 사용 금지라고 하지만 성격상 테스트를 너무 해보고 싶어서 집에서 해봤어요,국내 금지
...,...,...
1350,이거 하나 면 든든하게 세계 일주도 가능합니다,세계 일주
1351,모양도 이쁘고 휴대성도 좋습니다,모양 휴대
1352,깔끔한 디자인에 깔끔한 제품 마무리가 잘 된 어텝터네요,디자인 마무리 텝
1353,아직 국내라서 사용 전이지만 제품이 아담하고 케이스가 있어서 이쁜 듯,국내 케이스


In [22]:
sample = Dataset.from_pandas(sample)

In [23]:
sample.save_to_disk('/EEVE/sample')

Saving the dataset (0/1 shards):   0%|          | 0/1355 [00:00<?, ? examples/s]

In [24]:
dataset_path = "EEVE/sample"
dataset = load_from_disk(dataset_path)

In [36]:
dataset

Dataset({
    features: ['add_rule_based_split_review', 'Tokenized_morphs', '__index_level_0__'],
    num_rows: 1355
})

In [25]:
# NF4 양자화를 위한 설정
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True, # 모델을 4비트 정밀도로 로드
    bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
    bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
    bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=nf4_config,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [35]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40960, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [37]:
prompt_input_template = """
아래는 리뷰 데이터와 그에 대한 요약문이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.
### 리뷰 데이터:
{add_rule_based_split_review}
### 입력:
{input}
### 요약문:
"""
prompt_no_input_template = """
아래는 리뷰 데이터와 그에 대한 요약문이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.
### 리뷰 데이터:
{add_rule_based_split_review}
### 요약문:
"""

def generate_prompt(data_point):
    add_rule_based_split_review = data_point['add_rule_based_split_review']
    input_text = data_point.get('input', '')
    label = data_point.get('Tokenized_morphs', '')

    if input_text:
        res = prompt_input_template.format(add_rule_based_split_review=add_rule_based_split_review, input=input_text)
    else:
        res = prompt_no_input_template.format(add_rule_based_split_review=add_rule_based_split_review)

    if label:
        res = f'{res} {label}<lim_end>|'  # eos token 마지막 추가

    data_point['text'] = res
    return data_point


In [38]:
# 데이터셋에 프롬프트 적용
remove_column_keys = dataset.features.keys() 
dataset_cvted = dataset.shuffle().map(generate_prompt, remove_columns=remove_column_keys)

Map:   0%|          | 0/1355 [00:00<?, ? examples/s]

In [39]:
def tokenize_function(examples):
  outputs = tokenizer(examples["text"], truncation=True, max_length=512)
  return outputs

In [40]:
remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

Map:   0%|          | 0/1355 [00:00<?, ? examples/s]

In [41]:
lora_config = LoraConfig(
    r=4, # LoRA 가중치 행렬의 rank. 정수형이며 값이 작을수록 trainable parameter가 적어짐
    lora_alpha=8, # LoRA 스케일링 팩터. 추론 시 PLM weight와 합칠 때 LoRA weight의 스케일을 일정하게 유지하기 위해 사용
    lora_dropout=0.05,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], # LoRA를 적용할 layer. 모델 아키텍처에 따라 달라짐
    bias='none', # bias 파라미터를 학습시킬지 지정. ['none', 'all', 'lora_only']
    task_type=TaskType.CAUSAL_LM
)

# 양자화된 모델을 학습하기 전, 전처리를 위해 호출
model = prepare_model_for_kbit_training(model)
# LoRA 학습을 위해서는 아래와 같이 peft를 사용하여 모델을 wrapping 해주어야 함
model = get_peft_model(model, lora_config)

# 학습 파라미터 확인
model.print_trainable_parameters()

trainable params: 15,728,640 || all params: 10,820,653,056 || trainable%: 0.1454


In [42]:
# Data Collator 역할
# 각 입력 시퀀스의 input_ids(토큰) 길이를 계산하고, 가장 긴 길이를 기준으로 길이가 짧은 시퀀스에는 패딩 토큰 추가
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [43]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=2, # 각 디바이스당 배치 사이즈. 작을수록(1~2) 좀 더 빠르게 alignment 됨
    gradient_accumulation_steps=4, 
    warmup_steps=1,
    #num_train_epochs=1,
    max_steps=1000, 
    learning_rate=2e-4, # 학습률
    bf16=True, # bf16 사용 (지원되는 하드웨어 확인 필요)
    output_dir="outputs",
    optim="paged_adamw_8bit", # 8비트 AdamW 옵티마이저
    logging_steps=50, # 로깅 빈도
    save_total_limit=3 # 저장할 체크포인트의 최대 수
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_tokenized,
    max_seq_length=512, # 최대 시퀀스 길이
    args=train_args,
    dataset_text_field="text",
    data_collator=collate_fn
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` a

In [ ]:
# from transformers import Trainer, TrainingArguments
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from trl import SFTConfig, SFTTrainer
# from accelerate import Accelerator, DataLoaderConfiguration

# # SFTConfig 설정
# sft_config = SFTConfig(
#     max_seq_length=128,
#     dataset_text_field='text',
# )

# # Tokenizer 설정
# tokenizer = AutoTokenizer.from_pretrained('your-model-name')
# tokenizer.padding_side = 'right'

# # 모델 설정
# model = AutoModelForSequenceClassification.from_pretrained('your-model-name')

# # TrainingArguments 설정
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     save_steps=10_000,
#     save_total_limit=2,
# )

# # SFTTrainer 설정
# trainer = SFTTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset_cvted,  # 변환된 데이터셋 사용
#     tokenizer=tokenizer,
#     sft_config=sft_config,
# )

# # Accelerator 설정
# accelerator = Accelerator(
#     dataloader_config=DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
# )

# # 훈련 시작
# trainer.train()


In [44]:
model.config.use_cache = False
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,0.991900
100,0.636500
150,0.597900
200,0.551700
250,0.507600
300,0.494600
350,0.474000
400,0.365300
450,0.360600
500,0.363900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

TrainOutput(global_step=1000, training_loss=0.3625944323539734, metrics={'train_runtime': 4565.5373, 'train_samples_per_second': 1.752, 'train_steps_per_second': 0.219, 'total_flos': 3.67140887199744e+16, 'train_loss': 0.3625944323539734, 'epoch': 5.9})

In [45]:
FINETUNED_MODEL = "eeve_qlora"
trainer.model.save_pretrained(FINETUNED_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


QLoRA model loading

In [48]:
from peft import PeftConfig

FINETUNED_MODEL = "eeve_qlora"
peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [49]:
# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=nf4_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [50]:
# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [51]:
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Finetuning model inference test

In [57]:
prompt = """

#ROLE
너는 문장을 요악하는 사람이야

#명령문
문장을 입력받으면 그 문장을 두, 세 단어로 요약해줘

#입력문
USB형 여행용 어댑터 완전 저렴하게 잘 샀어요. C타입이 되서 매우 편리해요.


"""

# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=256)
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.92,
    repetition_penalty=1.2,
    add_special_tokens=True 
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


#출력문
여행용 어댑터 C타입 편리함


#참고자료
https://github.com/hongmin0314/summarization-with-BERT
